# Mixture of Experts (MoE) - Conceptos Fundamentales

Este notebook introduce los conceptos clave de la arquitectura Mixture of Experts (MoE) y su implementación en el proyecto.

In [ ]:
import sys
sys.path.append('..')

from moe_variants.switched_moe import SwitchedMoE
from utils.dashboard import ComplexityDashboard
import numpy as np
import matplotlib.pyplot as plt

## 1. Arquitectura MoE

### Componentes Principales:
1. Expertos: Modelos especializados en diferentes aspectos de los datos
2. Router: Sistema que dirige las entradas al experto más apropiado
3. Combinador: Integra las salidas de los expertos (en algunas variantes)

En este proyecto usamos una variante 'switched', donde cada entrada se dirige a un único experto.

In [ ]:
# Inicializar MoE y Dashboard
moe = SwitchedMoE(num_experts=3, complexity_threshold=0.6)
dashboard = ComplexityDashboard()

# Mostrar configuración
metrics = moe.get_metrics()
print("Configuración del MoE:")
print(f"Número de expertos: {moe.num_experts}")
print(f"Umbral de complejidad: {metrics['complexity_threshold']}")

print("\nDescripción de expertos:")
for expert_id, description in metrics['expert_descriptions'].items():
    print(f"Expert {expert_id}: {description}")

## 2. Análisis de Complejidad

El MoE utiliza métricas de complejidad para decidir qué experto debe procesar cada entrada.

In [ ]:
def analyze_complexity(inputs):
    results = []
    for input_data in inputs:
        complexity = moe._compute_complexity(input_data)
        expert = moe.route(input_data)
        confidence = 1.0 - min([
            abs(complexity - t) 
            for t in [metrics['complexity_threshold']*0.33, 
                     metrics['complexity_threshold']*0.66]
        ]) / metrics['complexity_threshold']
        
        results.append({
            'input': input_data,
            'complexity': complexity,
            'expert': expert,
            'confidence': confidence
        })
        
        # Actualizar dashboard
        dashboard.update_metrics('text_complexity', complexity)
        dashboard.update_metrics('expert_assignment', expert)
    
    return results

# Ejemplos con diferentes niveles de complejidad
test_inputs = [
    "The",                     # Simple
    "Python",                  # Medio
    "Programming",             # Medio-Alto
    "AI and ML",              # Medio con tokens cortos
    "@#$%^&*",                # Alto por caracteres especiales
    "Supercalifragilistic",   # Alto por longitud
]

results = analyze_complexity(test_inputs)

# Visualizar resultados
complexities = [r['complexity'] for r in results]
experts = [r['expert'] for r in results]
confidences = [r['confidence'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de complejidad vs experto
scatter = ax1.scatter(complexities, experts, c=confidences, 
                     cmap='viridis', s=100)
ax1.set_xlabel('Complejidad')
ax1.set_ylabel('Experto Asignado')
ax1.set_title('Asignación de Expertos por Complejidad')
plt.colorbar(scatter, ax=ax1, label='Confianza')

# Distribución de complejidad
ax2.hist(complexities, bins=10)
ax2.axvline(metrics['complexity_threshold']*0.33, 
            color='r', linestyle='--', label='Umbral Simple-Medio')
ax2.axvline(metrics['complexity_threshold']*0.66, 
            color='g', linestyle='--', label='Umbral Medio-Complejo')
ax2.set_xlabel('Complejidad')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Complejidad')
ax2.legend()

plt.tight_layout()
plt.show()

# Mostrar detalles
print("\nAnálisis detallado:")
for r in results:
    print(f"\nEntrada: {r['input']}")
    print(f"Complejidad: {r['complexity']:.2f}")
    print(f"Experto asignado: {r['expert']}")
    print(f"Confianza: {r['confidence']:.2f}")

## 3. Monitoreo en Tiempo Real

El proyecto incluye un dashboard interactivo para monitorear el comportamiento del MoE.

In [ ]:
# Simular procesamiento continuo
def simulate_continuous_processing(num_samples=50):
    chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789@#$%'
    
    for _ in range(num_samples):
        # Generar entrada aleatoria
        length = np.random.randint(1, 20)
        text = ''.join(np.random.choice(list(chars), size=length))
        
        # Procesar y actualizar métricas
        complexity = moe._compute_complexity(text)
        expert = moe.route(text)
        
        dashboard.update_metrics('text_complexity', complexity)
        dashboard.update_metrics('expert_assignment', expert)
        dashboard.update_metrics('processing_time', np.random.random() * 0.01)
        
        plt.pause(0.1)  # Breve pausa para visualización

# Iniciar simulación
simulate_continuous_processing()

## 4. Conclusiones

El MoE demuestra:
1. Especialización efectiva basada en complejidad
2. Asignación consistente de expertos
3. Alto nivel de confianza en las decisiones

Esta arquitectura es especialmente útil en:
- Procesamiento de lenguaje natural (LLMs)
- Análisis de imágenes (ViT)
- Tareas que requieren diferentes niveles de especialización